In [ ]:
import JSON
using FilePaths; using FilePathsBase: /

DATA_PATH = abspath(p"../out")
if !isdir(DATA_PATH)
    throw(ArgumentError("Data path does not exist"))
end
println(DATA_PATH)

INFO_FILE = DATA_PATH / "extra/non-terminating.json"


data_json = JSON.parsefile(string(INFO_FILE))
data_json

In [ ]:
MaybeInt = Union{Int, Nothing}
Intervals = Dict{String, MaybeInt}

function getintervals_bound(data_json::Dict, extra::String)::Intervals
    intervals = Dict()
    for (j1, dstIntervals) ∈ data_json
        if !isa(dstIntervals, Dict)
            throw(ArgumentError("dstIntervals is not a Dict"))
        end
        for (j2, it) ∈ dstIntervals
            intervals["$extra$j1-$j2-"] = it[1]
            intervals["$extra$j1-$j2+"] = it[2] < 0 ? nothing : it[2]
        end
    end
    intervals
end

function getintervals_module(moduleId::Int, data_json::Dict)::Intervals
    intervals = Dict()


    merge!(intervals, getintervals_bound(data_json["in"], "$(extra)I"))
    merge!(intervals, getintervals_bound(data_json["out"], "$(extra)O"))
    intervals
end

function getintervals_iteration(data_json::Dict)::Intervals
    intervals = Dict()
    modules = length(data_json)
    for (moduleId, modIntervals) in data_json
        mId = parse(Int, moduleId)

        if mId > 0
            merge!(intervals, getintervals_bound(modIntervals["in"], "$(moduleId)I"))
        end
        if mId + 1 < modules
            merge!(intervals, getintervals_bound(modIntervals["out"], "$(moduleId)O"))
        end
    end
    intervals
end

function getintervals(data_json::Dict)::Dict{String, Vector{MaybeInt}}
    if "intervals" ∉ keys(data_json)
        throw(ArgumentError("intervals not found"))
    end

    intervals = Dict{String,Vector{MaybeInt}}()
    data_intervals = data_json["intervals"]
    for value ∈ data_intervals
        iterintervals = getintervals_iteration(value)

        for (k, v) ∈ iterintervals
            vector = get!(intervals, k, Vector{MaybeInt}())
            push!(vector, v)
        end
    end

    # Clean columns that are all nothing
    toremove = Vector{String}()
    for (k, v) ∈ intervals
        if all(isnothing, v)
            push!(toremove, k)
        end
    end
    for k ∈ toremove
        delete!(intervals, k)
    end

    intervals
end


intervals = getintervals(data_json);

In [ ]:
using DataFrames

data = DataFrame(intervals)
data

In [ ]:
using StatsPlots
using Plots
plotlyjs()
default(size=(1000, 600))

df = data[!, length.(unique.(eachcol(data))) .> 1]
display(@df df plot(collect(axes(df, 1)), cols(), xlabel="Iteration", ylabel="Value", 
    legend=:topleft, title="Bounds over time"))
outfile = DATA_PATH / "figs/bounds/value-time.html"
mkdir(dirname(outfile), exist_ok=true)
Plots.savefig(outfile);
